# Selection de variables avec le lasso

In [2]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import math
import numpy as np
import re
import matplotlib.pyplot as plt
import pprint
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
from sklearn import svm
import numpy as np
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC 
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [4]:
from sklearn.linear_model import Lasso
#voir si besoin de standardiser les features


In [19]:
#importation des donnees
url_retards ='https://ressources.data.sncf.com/api/v2/catalog/datasets/regularite-mensuelle-tgv-aqst/exports/json?limit=-1&offset=0&timezone=UTC'
retards = requests.get(url_retards)
wb_retards = retards.json()

df_retards = pd.DataFrame(wb_retards)
df_retards.head(2)

,date,service,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,nb_annulation,commentaire_annulation,nb_train_depart_retard,retard_moyen_depart,...,nb_train_retard_sup_15,retard_moyen_trains_retard_sup15,nb_train_retard_sup_30,nb_train_retard_sup_60,prct_cause_externe,prct_cause_infra,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs
0,2018-01,National,LE MANS,PARIS MONTPARNASSE,56,406,1,None,213,8.479969,...,32,5.363539,9,4,20.000000,35.000000,16.666667,16.666667,8.333333,3.333333
1,2018-01,National,NANTES,PARIS MONTPARNASSE,128,525,3,None,193,7.294991,...,56,7.225734,29,11,37.333333,26.666667,21.333333,4.000000,9.333333,1.333333


In [20]:
#ajout de la colonne 'nombre de retards'
df_retards['nb_retards'] = df_retards['nb_train_retard_sup_15']+df_retards['nb_train_retard_sup_30']+df_retards['nb_train_retard_sup_60']


In [21]:
#pour homogeneiser les donnees, passage au pourcentage
df_retards['prct_nb_train_depart_retard'] = df_retards['nb_train_depart_retard']/df_retards['nb_train_prevu']*100
df_retards['prct_nb_annulation'] = df_retards['nb_annulation']/df_retards['nb_train_prevu']*100
df_retards[['annee','mois']]=df_retards.date.str.split('-',expand=True)
df_retards.head(2)

,date,service,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,nb_annulation,commentaire_annulation,nb_train_depart_retard,retard_moyen_depart,...,prct_cause_infra,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs,nb_retards,prct_nb_train_depart_retard,prct_nb_annulation,annee,mois
0,2018-01,National,LE MANS,PARIS MONTPARNASSE,56,406,1,None,213,8.479969,...,35.000000,16.666667,16.666667,8.333333,3.333333,45,52.463054,0.246305,2018,01
1,2018-01,National,NANTES,PARIS MONTPARNASSE,128,525,3,None,193,7.294991,...,26.666667,21.333333,4.000000,9.333333,1.333333,96,36.761905,0.571429,2018,01


In [22]:
#on retire les colonnes de commentaires qui sont generalement vides
df_retards = df_retards.drop('commentaire_annulation', axis = 1)
df_retards = df_retards.drop('commentaire_retards_depart', axis = 1)
df_retards = df_retards.drop('commentaires_retard_arrivee', axis = 1)
df_retards['annee'] = pd.to_numeric(df_retards['annee'])
df_retards['mois'] = pd.to_numeric(df_retards['mois'])

In [23]:
#on retire aussi les colonnes de NaN et dont les retards valent 0 qui ne nous interessent pas ici
df_retards.dropna(inplace = True)
df_retards = df_retards.drop(df_retards[df_retards['retard_moyen_depart']==0].index)

In [24]:
#on somme à présent les données par mois
df_retards = df_retards.groupby(['date'])[['duree_moyenne','prct_cause_materiel_roulant', 'prct_cause_infra','prct_cause_gestion_trafic','nb_train_prevu', 'prct_cause_externe','prct_cause_gestion_gare', 'prct_cause_prise_en_charge_voyageurs', 'nb_retards']].mean()
df_retards.head()

,duree_moyenne,prct_cause_materiel_roulant,prct_cause_infra,prct_cause_gestion_trafic,nb_train_prevu,prct_cause_externe,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs,nb_retards
date,,,,,,,,,
2018-01,171.604651,19.214568,29.355512,18.146421,273.093023,26.231441,4.276215,2.775842,44.023256
2018-02,173.284615,18.810722,19.461879,17.260284,249.376923,40.271564,3.250002,0.945549,62.823077
2018-03,173.623077,21.169229,19.066123,19.553561,272.753846,31.580144,4.951599,2.140882,41.861538
2018-04,170.666667,22.935976,21.747998,17.373356,280.759690,25.894618,7.439501,3.058163,30.937984
2018-05,158.698413,18.947225,15.841131,13.072013,285.531746,30.202739,5.688875,4.343255,41.436508


In [25]:
causes_retards = ['prct_cause_materiel_roulant', 'prct_cause_infra','prct_cause_gestion_trafic','nb_train_prevu', 'prct_cause_externe','prct_cause_gestion_gare', 'prct_cause_prise_en_charge_voyageurs']

In [26]:
x = sm.add_constant(df_retards[causes_retards])
y = df_retards[['nb_retards']].values.ravel()

xTrain, xTest, yTrain, yTest = train_test_split(
    x,y, test_size=0.2, random_state=0
)

In [27]:
features = df_retards[causes_retards]
features = StandardScaler().fit(features.values).transform(features.values)
df_retards[causes_retards] = features

In [28]:
#on fait la regression lineaire

results = sm.OLS(y, x).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.556
Model:                            OLS   Adj. R-squared:                  0.491
Method:                 Least Squares   F-statistic:                     8.570
Date:                Wed, 14 Dec 2022   Prob (F-statistic):           8.91e-07
Time:                        12:28:01   Log-Likelihood:                -219.32
No. Observations:                  56   AIC:                             454.6
Df Residuals:                      48   BIC:                             470.9
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [29]:
#3. Estimer un modèle LASSO et afficher les valeurs des coefficients
lasso1 = Lasso(fit_intercept=True,normalize=False, alpha = 0.1).fit(xTrain,yTrain)

#np.abs(lasso1.coef_)
features_selec = x.columns[np.abs(lasso1.coef_)>0].tolist()
features_selec

/opt/mamba/lib/python3.10/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


['prct_cause_materiel_roulant',
 'prct_cause_infra',
 'prct_cause_gestion_trafic',
 'nb_train_prevu',
 'prct_cause_externe',
 'prct_cause_gestion_gare',
 'prct_cause_prise_en_charge_voyageurs']

In [30]:
#4. Corrélations entre les variables sélectionnées
corr = df_retards[features_selec].corr()

plt.figure()
p = corr.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)
p

/tmp/ipykernel_1171/3579470952.py:5: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  p = corr.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)


,prct_cause_materiel_roulant,prct_cause_infra,prct_cause_gestion_trafic,nb_train_prevu,prct_cause_externe,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs
prct_cause_materiel_roulant,1.00,-0.54,0.61,0.56,-0.13,0.04,-0.16
prct_cause_infra,-0.54,1.00,-0.31,-0.73,-0.31,-0.42,-0.31
prct_cause_gestion_trafic,0.61,-0.31,1.00,0.36,-0.11,-0.08,-0.42
nb_train_prevu,0.56,-0.73,0.36,1.00,0.02,0.36,0.26
prct_cause_externe,-0.13,-0.31,-0.11,0.02,1.00,-0.18,-0.33
prct_cause_gestion_gare,0.04,-0.42,-0.08,0.36,-0.18,1.00,0.54
prct_cause_prise_en_charge_voyageurs,-0.16,-0.31,-0.42,0.26,-0.33,0.54,1.00


<Figure size 640x480 with 0 Axes>

In [31]:
#5. Faire une régression linéaire parcimonieuse et non parcimonieuse

## Régression parcimonieuse
yPred_parci = lasso1.predict(xTest)
rmse_parci = sklearn.metrics.mean_squared_error(yTest, yPred_parci, squared = False)
print("rmse_parci : ", rmse_parci)
rsq_parci = sklearn.metrics.r2_score(yTest, yPred_parci) 
print("rsq_parci : ", rsq_parci)

## Régression non parcimonieuse
ols = LinearRegression().fit(xTrain, yTrain)
yPred_nonparci = ols.predict(xTest)
rmse_nonparci = sklearn.metrics.mean_squared_error(yTest, yPred_nonparci, squared = False)
print("rmse_nonparci : ", rmse_nonparci)
rsq_nonparci = sklearn.metrics.r2_score(yTest, yPred_nonparci) 
print("rsq_nonparci : ", rsq_nonparci)

# Le modèle non parcimonieux est (légèrement) plus performant. 

rmse_parci :  11.204042457528042
rsq_parci :  -0.5443569417082845
rmse_nonparci :  11.356294836452454
rsq_nonparci :  -0.5866148375449707
